### Chatbot with Conversational History

In [48]:
import os
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv('GROQ_API_KEY')

In [49]:
from langchain_groq import ChatGroq
model = ChatGroq(model='Gemma2-9b-It', groq_api_key=groq_api_key)
model

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000014A285151D0>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014A28515E50>, model_name='Gemma2-9b-It', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [50]:
from langchain_core.prompts import ChatPromptTemplate
prompt = ChatPromptTemplate.from_messages([
    ('user', 'Hi my name is Yuvraj and Im a GenAI engineer'),
    ("ai", "Hi Yuvraj, its nice to meet you. Thats fascinating, being a GenAI engineer is cutting edge and important work. What kind of projects are you working on? Im always eager to learn more about the advancements in the field.  Perhaps you could tell me about some of the challenges and rewards you encounter in your role?"),
    ('user', "Whats my name and what do i do?")
])

chain=prompt|model
chain.invoke({})

AIMessage(content='You are Yuvraj, and you are a GenAI engineer!  \n\nIt seems I have a good memory for introductions. 😊  What can I help you with today?\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 39, 'prompt_tokens': 108, 'total_tokens': 147, 'completion_time': 0.070909091, 'prompt_time': 0.002811448, 'queue_time': 0.246072141, 'total_time': 0.073720539}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--1317288d-1b7e-4772-a366-c78e5a6af9d3-0', usage_metadata={'input_tokens': 108, 'output_tokens': 39, 'total_tokens': 147})

### Message History
We can use message history class to wrap our model and make it stateful. This will keep track of the inputs and outputs of the model and store them in some datastore. Future interactions will then load them and pass them into the chain as the input. Lets see how to use this!

In [51]:
from langchain_community.chat_message_histories import ChatMessageHistory
from langchain_core.chat_history import BaseChatMessageHistory
from langchain_core.runnables.history import RunnableWithMessageHistory

store={}

def get_session_history(session_id: str):
    if session_id not in store:
        store[session_id]=ChatMessageHistory()
    return store[session_id]

with_message_history=RunnableWithMessageHistory(model, get_session_history)

In [52]:
config={'configurable': {'session_id': 'chat1'}}

In [53]:
from langchain_core.messages import HumanMessage, AIMessage

response=with_message_history.invoke([
    HumanMessage(content='Hi my name is Yuvraj and Im GenAI Engineer')],
    config=config)

In [54]:
response.content

"Hi Yuvraj, it's nice to meet you! \n\nBeing a GenAI Engineer is fascinating. What are you working on these days?  Are there any particular projects or areas of GenAI that you're particularly excited about?\n"

In [55]:
with_message_history.invoke([
    HumanMessage(content='Hi, what is my name and what do i do?')],
    config=config)

AIMessage(content="You are Yuvraj and you are a GenAI Engineer. 😊  \n\nIs there anything else you'd like to know or talk about?\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 33, 'prompt_tokens': 94, 'total_tokens': 127, 'completion_time': 0.06, 'prompt_time': 0.002809718, 'queue_time': 0.253604841, 'total_time': 0.062809718}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--751a8f76-e37b-4621-815e-d00e550cc799-0', usage_metadata={'input_tokens': 94, 'output_tokens': 33, 'total_tokens': 127})

In [56]:
config1 = {'configurable': {'session_id': 'chat2'}}

In [57]:
# changed session id using new config
response=with_message_history.invoke([
    HumanMessage(content='Hi what is my name?')],
    config=config1)
response

AIMessage(content="As a large language model, I have no memory of past conversations and do not know your name. If you'd like to tell me your name, I'd be happy to use it! 😊\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 44, 'prompt_tokens': 15, 'total_tokens': 59, 'completion_time': 0.08, 'prompt_time': 0.001252549, 'queue_time': 0.24836557, 'total_time': 0.081252549}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--f2442149-e5b1-49b3-ba70-16101177181c-0', usage_metadata={'input_tokens': 15, 'output_tokens': 44, 'total_tokens': 59})

In [58]:
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions to best of your ability'),
    MessagesPlaceholder(variable_name='messages')
])

chain=prompt|model

In [59]:
chain.invoke({'messages': [HumanMessage(content='Hi my name is Yuvraj')]})

AIMessage(content="Hi Yuvraj! It's nice to meet you.  \n\nWhat can I do for you today? 😊  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 31, 'total_tokens': 60, 'completion_time': 0.052727273, 'prompt_time': 0.001481079, 'queue_time': 0.25645411, 'total_time': 0.054208352}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--7a542b3c-a6ca-4141-a545-7da670e1a330-0', usage_metadata={'input_tokens': 31, 'output_tokens': 29, 'total_tokens': 60})

In [60]:
with_message_history=RunnableWithMessageHistory(chain, get_session_history)
config={'configurable': {'session_id': 'chat3'}}

In [61]:
response=with_message_history.invoke([
    HumanMessage(content='Hi my name is yuvraj')],
    config=config
    )
response

AIMessage(content="Hello Yuvraj, it's nice to meet you!\n\nI'm happy to help with any questions you have. Just ask away! 😊  \n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 35, 'prompt_tokens': 31, 'total_tokens': 66, 'completion_time': 0.063636364, 'prompt_time': 0.001493069, 'queue_time': 0.25770135, 'total_time': 0.065129433}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--57da9b00-67fc-4e23-8109-75a50201da35-0', usage_metadata={'input_tokens': 31, 'output_tokens': 35, 'total_tokens': 66})

In [62]:
# adds more complexity
prompt = ChatPromptTemplate.from_messages([
    ('system', 'You are a helpful assistant. Answer all the questions to the best of your ability in {language}'),
    MessagesPlaceholder(variable_name='messages'),
])
chain = prompt|model

In [63]:
response=chain.invoke({'messages': [HumanMessage(content='Hi My name is Yuvraj')], 'language': 'Hindi'})
response

AIMessage(content="नमस्ते Yuvraj! \n\nमुझे बहुत खुशी है कि आप मुझसे बात करना चाहते हैं। आपसे क्या जानना चाहेंगे? \n\n(Hello Yuvraj! I'm very happy you want to talk to me. What would you like to know?) \n\n\nमुझे कोई भी प्रश्न पूछ सकते हैं, मैं अपनी पूरी कोशिश करूँगा कि इसका उत्तर दे सकूँ। \n\n(You can ask me any question, I will try my best to answer it.)\n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 112, 'prompt_tokens': 34, 'total_tokens': 146, 'completion_time': 0.203636364, 'prompt_time': 0.001487199, 'queue_time': 0.24810985, 'total_time': 0.205123563}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--b0658672-48ab-44ba-a154-21c47a9966d9-0', usage_metadata={'input_tokens': 34, 'output_tokens': 112, 'total_tokens': 146})

Lets now wrap the more complicated chain in MessageHistory class. This time because there are multiple keys in the input we need to specify the correct key to use to save the chat history

In [64]:
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages'
    
)

In [65]:
config={'configurable': {'session_id': 'chat4'}}
response = with_message_history.invoke({'messages': [HumanMessage(content='Hi my name is Yuvraj')], 'language': 'Hindi'},
                                       config=config)
response.content

'नमस्ते Yuvraj! 👋 \n\nमुझे बहुत खुशी हो कि आप मुझसे बात कर रहे हैं।  \n\nआप क्या पूछना चाहेंगे? 😊 \n\n'

### Managing Chat conversation history

In [66]:
from langchain_core.messages import SystemMessage, trim_messages
# trim_messages is a trimmer that allows to specify how many tokens we want to keep along with other parameters like if we want to always keep the system message and weather to allow partial messages

trimmer = trim_messages(
    max_tokens=40, # defines the llms context window size
    strategy='last',
    token_counter=model,
    include_system=True,
    allow_partial=False,
    start_on='human'
    
)

messages = [
    SystemMessage(content='You are a helpful assistant'),
    HumanMessage(content='Hi how are you'),
    AIMessage(content='Good'),
    HumanMessage(content='Hi who are you'),
    AIMessage(content='Im a AI assistant'),
    HumanMessage(content='I like choclate ice cream'),
    AIMessage(content='Oh that great'),
    HumanMessage(content='Hi again'),
    AIMessage(content='Hello again'),
]
trimmer.invoke(messages)

[SystemMessage(content='You are a helpful assistant', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='I like choclate ice cream', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Oh that great', additional_kwargs={}, response_metadata={}),
 HumanMessage(content='Hi again', additional_kwargs={}, response_metadata={}),
 AIMessage(content='Hello again', additional_kwargs={}, response_metadata={})]

In [67]:
from operator import itemgetter
from langchain_core.runnables import RunnablePassthrough

chain=(
    RunnablePassthrough.assign(messages=itemgetter('messages')|trimmer)
    |prompt
    |model
)

response=chain.invoke({
    'messages': messages + [HumanMessage(content='What ice cream do i like')],
    'language':'English'}
)
response

AIMessage(content='You like chocolate ice cream!  🍫  \n\nIs there a particular kind of chocolate ice cream you enjoy most?  😊 \n\n', additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 30, 'prompt_tokens': 73, 'total_tokens': 103, 'completion_time': 0.054545455, 'prompt_time': 0.002424558, 'queue_time': 0.257056192, 'total_time': 0.056970013}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--bc6d0f41-1bc7-4561-9d9c-dddb0606bc3d-0', usage_metadata={'input_tokens': 73, 'output_tokens': 30, 'total_tokens': 103})

In [68]:
# lets wrap this in message history
with_message_history=RunnableWithMessageHistory(
    chain,
    get_session_history,
    input_messages_key='messages'
)
config={'configurable': {'session_id': 'chat5'}}

In [69]:
response=chain.invoke({
    'messages': [HumanMessage(content='What ice cream do i like')],
    'language':'English'}
)
response

AIMessage(content="As an AI, I don't have personal preferences or memories, including your ice cream taste!  \n\nTo figure out what ice cream you like, think about:\n\n* **Your favorite flavors:** Chocolate, vanilla, strawberry, something more unique like pistachio or lavender?\n* **Your preferred texture:** Creamy, chunky with mix-ins, sorbet, or something else?\n* **Any toppings you love:** Sprinkles, nuts, whipped cream, fruit?\n\n\nTell me about your favorite flavors and I can give you some ice cream suggestions! 🍨🍦  \n\n", additional_kwargs={}, response_metadata={'token_usage': {'completion_tokens': 119, 'prompt_tokens': 33, 'total_tokens': 152, 'completion_time': 0.216363636, 'prompt_time': 0.001481749, 'queue_time': 0.24829959, 'total_time': 0.217845385}, 'model_name': 'Gemma2-9b-It', 'system_fingerprint': 'fp_10c08bf97d', 'service_tier': 'on_demand', 'finish_reason': 'stop', 'logprobs': None}, id='run--c1c6c7d1-b965-424a-9bc9-65e84e26fc08-0', usage_metadata={'input_tokens': 33,

### Setting up vector stores and retrievers

In [70]:
from langchain_core.documents import Document

documents = [
    Document(
        page_content='Dogs are great companions',
        meta_data={'source': 'mammal-pets-doc'}
    ),
    Document(
        page_content='Cats are independent pets',
        meta_data={'source': 'mammal-pets-doc'}
    ),
    Document(
        page_content='Goldfish are popular',
        meta_data={'source': 'fist-pets-doc'}
    ),
    Document(
        page_content='Parrots are intelligent birds',
        meta_data={'source': 'bird-pets-doc'}
    ),
    Document(
        page_content='Rabbits are fast',
        meta_data={'source': 'mammal-pets-doc'}
    ),
]

In [71]:
documents

[Document(metadata={}, page_content='Dogs are great companions'),
 Document(metadata={}, page_content='Cats are independent pets'),
 Document(metadata={}, page_content='Goldfish are popular'),
 Document(metadata={}, page_content='Parrots are intelligent birds'),
 Document(metadata={}, page_content='Rabbits are fast')]

In [83]:
import os
from langchain_groq import ChatGroq
from dotenv import load_dotenv
load_dotenv()

groq_api_key=os.getenv('GROQ_API_KEY')
os.environ['HUGGING_FACE_TOKEN']=os.getenv('HUGGING_FACE_TOKEN')

llm=ChatGroq(model='llama-3.1-8b-instant', groq_api_key=groq_api_key)
llm

ChatGroq(client=<groq.resources.chat.completions.Completions object at 0x0000014A282FB890>, async_client=<groq.resources.chat.completions.AsyncCompletions object at 0x0000014A22B77AD0>, model_name='llama-3.1-8b-instant', model_kwargs={}, groq_api_key=SecretStr('**********'))

In [73]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings=HuggingFaceEmbeddings(model='all-MiniLM-L6-v2')

In [74]:
# vector stores
from langchain_chroma import Chroma

vectorstore = Chroma.from_documents(
    documents=documents,
    embedding=embeddings)
vectorstore

In [75]:
vectorstore.similarity_search('Cats')

[Document(id='52f4df50-ceaa-4d29-9518-894c495d7b64', metadata={}, page_content='Cats are independent pets'),
 Document(id='94015632-ff01-44c1-8196-69fd418a52cf', metadata={}, page_content='Cats are independent pets'),
 Document(id='38e0a131-8b81-4bb4-a9d6-c0ff671131ef', metadata={}, page_content='Rabbits are fast'),
 Document(id='21a046fd-1b2d-4a98-b447-05f7570febfe', metadata={}, page_content='Rabbits are fast')]

In [76]:
# async query
await vectorstore.asimilarity_search('cats')

[Document(id='52f4df50-ceaa-4d29-9518-894c495d7b64', metadata={}, page_content='Cats are independent pets'),
 Document(id='94015632-ff01-44c1-8196-69fd418a52cf', metadata={}, page_content='Cats are independent pets'),
 Document(id='38e0a131-8b81-4bb4-a9d6-c0ff671131ef', metadata={}, page_content='Rabbits are fast'),
 Document(id='21a046fd-1b2d-4a98-b447-05f7570febfe', metadata={}, page_content='Rabbits are fast')]

In [77]:
from typing import List
from langchain_core.documents import Document
from langchain_core.runnables import RunnableLambda

retriever = RunnableLambda(vectorstore.similarity_search).bind(k=1)
retriever.batch(['cat', 'dog'])

[[Document(id='52f4df50-ceaa-4d29-9518-894c495d7b64', metadata={}, page_content='Cats are independent pets')],
 [Document(id='99016665-fc80-4771-88a8-f58087a49daf', metadata={}, page_content='Dogs are great companions')]]

In [78]:
retriever=vectorstore.as_retriever(
    search_type='similarity',
    search_kwargs={'k':1}
)
retriever.batch(['cat', 'dog'])

[[Document(id='52f4df50-ceaa-4d29-9518-894c495d7b64', metadata={}, page_content='Cats are independent pets')],
 [Document(id='99016665-fc80-4771-88a8-f58087a49daf', metadata={}, page_content='Dogs are great companions')]]

In [84]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnablePassthrough

message="""
Answer this question using the provuded context only. {question}
Context: {context}
"""

prompt=ChatPromptTemplate.from_messages([('human',message)])

rag_chain={'context': retriever, 'question': RunnablePassthrough()}|prompt|llm

response=rag_chain.invoke('Tell me about dogs')
response.content

"Based on the provided context, here's what I can tell you about dogs:\n\nDogs are great companions."